<a href="https://colab.research.google.com/github/Anirudho747/Edrk/blob/main/FinanceAdvisor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# ============================== INSTALLS ==============================
!pip -q install langgraph langchain langchain-groq gradio

# ============================== IMPORTS ===============================
import os, re, json
from typing import TypedDict, Optional, Dict, Any, List
from langgraph.graph import StateGraph
from langchain_groq import ChatGroq

# If you're in Colab and stored your key in "Variables" (userdata)
try:
    from google.colab import userdata  # will exist in Colab
except Exception:
    userdata = None

# ============================== CONFIG ================================
# Set key from Colab variables if not already set in env
if ("GROQ_API_KEY" not in os.environ or not os.environ["GROQ_API_KEY"]) and userdata is not None:
    os.environ["GROQ_API_KEY"] = userdata.get("GROQ_API_KEY")

if "GROQ_API_KEY" not in os.environ or not os.environ["GROQ_API_KEY"]:
    raise ValueError("Please set GROQ_API_KEY (Colab: Settings ▶ Variables ▶ add GROQ_API_KEY).")

# Choose a Groq model
llm = ChatGroq(model_name="llama-3.3-70b-versatile", groq_api_key=os.environ["GROQ_API_KEY"])

def ask_llm(prompt: str) -> str:
    """Simple helper to query the LLM and return clean text."""
    return llm.invoke(prompt).content.strip()

# ============================== STATE TYPE ============================
class BotState(TypedDict, total=False):
    user_input: str
    intent: Optional[str]           # expense | budget | advice | unknown
    data: Optional[str]
    expenses: List[Dict[str, Any]]
    hitl_flag: bool                 # safety flag

# ============================== VOCAB =================================
INTENT_KEYWORDS = {
    "budget": {
        "budget","summary","report","total spend","spending","how much spent",
        "overall","breakdown","by category","expense summary","show budget"
    },
    "expense": {
        # verbs
        "add","spent","spend","pay","paid","buy","bought","purchase","charge",
        # money tokens / symbols / currencies
        "rs","inr","₹","$","eur","usd","rupees","amount","cost","price",
        # phrases users often say
        "log this","note this","record this","expense"
    },
    "advice": {
        "advice","advise","suggest","suggestion","tip","tips","plan",
        "how do i","how to","save for","reduce","cut","lower","optimize"
    }
}

CATEGORY_VOCAB = {
    "groceries": ["grocery","groceries","supermarket","kirana","food store"],
    "rent": ["rent","lease"],
    "travel": ["travel","flight","flights","airfare","train","uber","ola","bus","taxi","cab","hotel"],
    "dining": ["dine","dining","restaurant","eat out","takeaway","swiggy","zomato","coffee","cafe"],
    "utilities": ["electricity","power","water","gas","utility","utilities","wifi","broadband","internet"],
    "phone": ["mobile","phone","recharge","prepaid","postpaid"],
    "fuel": ["fuel","petrol","diesel"],
    "shopping": ["shopping","amazon","flipkart","clothes","apparel","shoes"],
    "health": ["health","medical","medicine","pharmacy","chemist","hospital","doctor"],
    "education": ["tuition","fees","course","exam","books","college","school"],
    "insurance": ["insurance","premium"],
    "loan": ["emi","loan","mortgage","interest"],
    "entertainment": ["movie","ott","netflix","prime","disney","music","game","theatre"],
    "transport": ["metro","parking","toll","commute","transport","bus pass"]
}

# ============================== HELPERS ===============================
NUM_WORDS = {
    "k": 1_000,
    "thousand": 1_000,
    "lakh": 100_000,
    "lac": 100_000,
    "cr": 10_000_000,
    "crore": 10_000_000,
}

def _normalize_text(s: str) -> str:
    s = s.lower().strip()
    s = s.replace("rs.", "rs ").replace("₹", " rs ").replace("inr", " rs ")
    s = re.sub(r"\s+", " ", s)
    return s

def _parse_scaled_number(token: str) -> Optional[float]:
    """
    Supports: 1200, 1,200, 1.2k, 2k, 1 lakh, 2.5 lakh, 3 cr, etc.
    """
    token = token.replace(",", "").strip()
    # 1) pure number
    m = re.fullmatch(r"\d+(\.\d+)?", token)
    if m:
        return float(token)
    # 2) number + suffix like 1.2k, 2k, 1.5lakh, 3cr
    m = re.fullmatch(r"(\d+(\.\d+)?)(k|thousand|lakh|lac|cr|crore)", token)
    if m:
        val = float(m.group(1))
        mult = NUM_WORDS[m.group(3)]
        return val * mult
    return None

def parse_amount_and_category(text: str):
    """Return (amount, category) from a free-text message."""
    t = _normalize_text(text)

    # Try two-token patterns like "1.5 lakh", "2 cr"
    two_token = re.findall(r"(\d+(?:\.\d+)?)\s*(k|thousand|lakh|lac|cr|crore)", t)
    if two_token:
        val, suf = two_token[0]
        amt = float(val) * NUM_WORDS[suf]
    else:
        # else scan tokens for first numeric-ish token
        amt = None
        for tok in t.split():
            val = _parse_scaled_number(tok)
            if val is not None:
                amt = val
                break

    # Category detection
    cat = "general"
    for canonical, synonyms in CATEGORY_VOCAB.items():
        if any(word in t for word in synonyms):
            cat = canonical
            break

    return (amt if amt is not None else None), cat

def is_high_risk(tl: str) -> bool:
    risky_keywords = [
        "retirement","liquidate","loan against","pledge","sell house","sell my house",
        "quit job","all-in","bet everything","margin","mortgage my",
        "crypto all","withdraw provident fund","pf withdraw","withdraw epf","empty my savings",
        "take loan for stocks","loan for crypto","gold loan for trading"
    ]
    return any(k in tl for k in risky_keywords)

def _contains_any(t: str, bag) -> bool:
    return any(k in t for k in bag)

# Few-shot LLM fallback for intent only if rules fail
INTENT_FEWSHOT = """Classify the user's intent as one word: expense, budget, advice, or unknown.
User: "Add 1200 rent" -> expense
User: "How much did I spend?" -> budget
User: "Tips to reduce travel costs" -> advice
User: "Tell me a joke" -> unknown
User: "{q}" ->"""

def classify_intent_llm(q: str) -> str:
    resp = ask_llm(INTENT_FEWSHOT.format(q=q)).lower().strip()
    return resp if resp in {"expense","budget","advice"} else "unknown"

# ============================== NODES =================================
def node_intent(state: BotState) -> BotState:
    text = state.get("user_input", "")
    tl = _normalize_text(text)

    # 1) Safety first
    state["hitl_flag"] = is_high_risk(tl)

    # 2) Priority: advice > budget > expense
    if _contains_any(tl, INTENT_KEYWORDS["advice"]):
        intent = "advice"
    elif _contains_any(tl, INTENT_KEYWORDS["budget"]):
        intent = "budget"
    elif _contains_any(tl, INTENT_KEYWORDS["expense"]):
        intent = "expense"
    else:
        # 3) LLM fallback
        intent = classify_intent_llm(text)

    state["intent"] = intent
    state["data"]  = f"(intent={intent}, hitl={state['hitl_flag']})"
    return state

def node_expense(state: BotState) -> BotState:
    amt, cat = parse_amount_and_category(state["user_input"])
    if amt is None:
        state["data"] = "Please say an amount, e.g., 'Add 50 groceries'."
        return state
    state.setdefault("expenses", []).append({"amount": amt, "category": cat})
    state["data"] = f"✅ Added expense: {amt} for {cat}."
    return state

def node_budget(state: BotState) -> BotState:
    exps = state.get("expenses", [])
    total = sum(e["amount"] for e in exps) if exps else 0
    by_cat: Dict[str, float] = {}
    for e in exps:
        by_cat[e["category"]] = by_cat.get(e["category"], 0) + e["amount"]
    state["data"] = json.dumps({"total_spent": total, "by_category": by_cat}, ensure_ascii=False)
    return state

def node_advice(state: BotState) -> BotState:
    text = state["user_input"]
    prompt = f"""Give exactly 3 short, friendly money tips for this request (no jargon, numbered 1-3):
User: "{text}" """
    state["data"] = ask_llm(prompt)
    return state

def node_hitl(state: BotState) -> BotState:
    state["data"] = (
        "⚠️ This looks high-risk. Please consult a certified financial advisor before acting. "
        "I’ll pause here until a human reviews your request."
    )
    return state

def node_fallback(state: BotState) -> BotState:
    state["data"] = "I can help with: expenses (e.g., 'Add 50 groceries'), budget, and advice."
    return state

# ============================== ROUTER ================================
def choose_next(state: BotState) -> str:
    if state.get("hitl_flag"):
        return "hitl"
    intent = state.get("intent", "unknown")
    if intent == "expense": return "expense"
    if intent == "budget":  return "budget"
    if intent == "advice":  return "advice"
    return "fallback"

# ============================== GRAPH BUILD ===========================
builder = StateGraph(BotState)
builder.add_node("Intent",   node_intent)
builder.add_node("expense",  node_expense)
builder.add_node("budget",   node_budget)
builder.add_node("advice",   node_advice)
builder.add_node("hitl",     node_hitl)
builder.add_node("fallback", node_fallback)

builder.set_entry_point("Intent")
builder.add_conditional_edges(
    "Intent",
    choose_next,
    {"hitl":"hitl","expense":"expense","budget":"budget","advice":"advice","fallback":"fallback"},
)
graph = builder.compile()

# ============================== REPL CHAT ============================
def run_chat():
    print("💬 Personal Finance Bot (type 'exit' to quit)")
    state: BotState = {"expenses": [], "hitl_flag": False}
    while True:
        msg = input("You: ").strip()
        if msg.lower() in ("exit", "quit"):
            print("Bot: Bye! 👋")
            break
        state["user_input"] = msg
        out = graph.invoke(state)
        print("Bot:", out.get("data", ""))
        state = out  # persist across turns

# Uncomment to use terminal-style chat:
#run_chat()